# Gradual Typing


- optional: type checker doesn't emit warning. It assumes the Any type when it cannot determine
  the type of an object. The Any type is considered compatible with all other types
- Does not catch type errors at runtime
- Does not enhance performance: In theory type annotations are like labels you can put on your code
  to tell what kind of data should go where.
  However, as of July 2021, no version of Python actually uses these labels to speed up your code.

Note that: Type hints are optional at all levels: you can have entire packages with no type hints

$\color{red}{warning}$: it’s better not to use type hints every where. For example, if adding type hints would make a function harder to use (less “user-friendly”) or would make the code more complicated than it needs to be, it’s better to leave them out


In [1]:
# Example 8-1
# without type hints
def show_count(count, word):
    if count == 1:
        return f"1 {word}"
    count_str = str(count) if count else "no"
    return f"{count_str} {word}s"


print(show_count(99, "bird"))
print(show_count(1, "bird"))

99 birds
1 bird


In [6]:
from pytest import mark


#!pip install mypy
# possibility of different results because occasional changes in it break backward compatibility.
# Example 8-2
@mark.parametrize(
    "qty, expected",
    [
        (1, "1 part"),
        (2, "2 parts"),
    ],
)
def test_show_count(qty, expected):
    got = show_count(qty, "part")
    assert got == expected


def test_show_count_zero():
    got = show_count(0, "part")
    assert got == "no parts"

- --disallow-untyped-defs makes Mypy report an error for any function that does not have type hints for all its parameters

- --disallow-incomplete-defs makes Mypy report an error for any function that has incomplete type hints. Incomplete type hints are type hints that are missing some information, such as the type of some parameters or the return value

$\color{red}{Note}$ $\color{red}{that}$

Using --disallow-untyped-defs on messages_test.py produces <u>three errors and a note</u>, because none of the functions in messages.py and messages_test.py have type hints.

Using --disallow-incomplete-defs on messages_test.py produces <u>no errors</u>, because none of the functions have any type hints

**Another way is using mypy.ini**

[mypy]

python_version = 3.9

warn_unused_configs = True

disallow_incomplete_defs = True


In [ ]:
def show_count(count: int, word: str) -> str:
    pass

In [ ]:
# Example 8-3.
def show_count(count: int, singular: str, plural: str = "") -> str:
    if count == 1:
        return f"1 {singular}"
    count_str = str(count) if count else "no"
    if not plural:
        plural = singular + "s"
    return f"{count_str} {plural}"


# fixing type error in message_test file in result of adding this function
def test_irregular() -> None:
    got = show_count(2, "child", "children")
    assert got == "2 children"

def hex2rgb(color=str) -> tuple[int, int, int]:

$\color{red}{error}$ : Function is missing a type annotation for one or more arguments


replace for Example 8-3

In this example if you don’t assign a default value to plural, the Python runtime will treat it as a required parameter.

$\color{green}{Optional[str]}$ just means: the type of this parameter may be str or NoneType


In [ ]:
from typing import Optional


def show_count(count: int, singular: str, plural: Optional[str] = None) -> str:
    pass

# Types Are Defined by Supported Operations


Note that: Instead of looking at the representation or the value space of a data type, it is more useful to look at the set of supported operations as the defining characteristic of a data type.


Now assume these two examples:
x can be (int, complex, Fraction, numpy.uint32, etc.) or (str, tuple, list, array)
or any type the inherits a `mul` method

```python
def double(x):
    return x * 2
```

A type checker will reject below code. If you tell Mypy that x is of type abc.Sequence, it
will flag x \* 2 as an error because the Sequence ABC does not implement or inherit
the `mul` method.

```python
def double(x: collections.abc.Sequence):
    return x * 2
```


**two different views of types**

- `Duck typing:` It doesn’t matter what the declared type of the object is, only what operations it actually supports. The name comes from the saying “If it walks like a duck and quacks like a duck, then it must be a duck”

- `Nominal typing:` Check the types of the objects and variables before running the code, and require that the types match exactly or be compatible. The type checker will only look at the names or declarations of the types, not their structures or behaviors


`duck typing vs nominal typing` <br>
duck typing is more flexible at the cost of allowing more errors at the run time while
nominal typing is more rigid with the advantage of catching some bugs earlier in a build pipeline, or even as the code is typed in an IDE.


Mypy sees that alert_bird is problematic: the type
hint declares the birdie parameter with type Bird, but the body of the function calls
birdie.quack()—and the Bird class has no such method.


In [ ]:
# Example 8-4
class Bird:
    pass


class Duck(Bird):
    def quack(self):
        print("Quack!")


def alert(birdie):
    birdie.quack()


def alert_duck(birdie: Duck) -> None:
    birdie.quack()


def alert_bird(birdie: Bird) -> None:
    birdie.quack()


# Example 8-5.
daffy = Duck()
alert(daffy)
alert_duck(daffy)
alert_bird(daffy)

Quack!
Quack!
Quack!


In [ ]:
# Example 8-6
woody = Bird()
alert(woody)
alert_duck(
    woody
)  # Mypy reported the problem: Argument 1 to "alert_duck" has incompatible type "Bird"; expected "Duck".
alert_bird(woody)

# Mypy errors
""" …/birds/ $ mypy woody.py
birds.py:16: error: "Bird" has no attribute "quack"
woody.py:5: error: Argument 1 to "alert_duck" has incompatible type "Bird";
expected "Duck"
Found 2 errors in 2 files (checked 1 source file) """

AttributeError: 'Bird' object has no attribute 'quack'

- Mypy could not detect this error because there are no type hints in alert.
- Mypy has been telling us since Example 8-4 that the body of the alert_bird
  function is wrong: "Bird" has no attribute "quack".


This little experiment shows that duck typing is easier to get started and is more flexible,
but allows unsupported operations to cause errors at runtime. Nominal typing
detects errors before runtime, but sometimes can reject code that actually runs—such
as the call alert_bird(daffy)


In [ ]:
# Example 8-7.
woody = Bird()
# same error for all of them
alert(woody)
alert_duck(woody)
alert_bird(woody)

AttributeError: 'Bird' object has no attribute 'quack'

# Types Usable in Annotations


This section covers all the major types you can use with annotations:

- typing.Any
- Simple types and classes
- typing.Optional and typing.Union
- Generic collections, including tuples and mappings
- Abstract base classes
- Generic iterables
- Parameterized generics and TypeVar
- typing.Protocols—the key to static duck typing
- typing.Callable
- typing.NoReturn—a good way to end this list


## The Any Type


compatiable with all types, can be used when you don’t know or don’t care about the type of a value

When a type checker sees an untyped function like this:

def double(x):

        return x * 2

it assumes this:

def double(x: Any) -> Any:

        return x * 2


object is like any but in contrast the type checker rejects it becuase does not support the ** mul ** operation

def double(x: object) -> object:

        return x * 2

More general types have narrower interfaces, i.e., they support fewer operations.Any accepts values of every type and the most specialized type, supporting every possible operation. At
least, that’s how the type checker understands Any.

Of course, no type can support every possible operation, so using Any prevents the
type checker from fulfilling its core mission: detecting potentially illegal operations
before your program crashes with a runtime exception.


### Subtype-of versus consistent-with


If you have a class T1 and a subclass T2, then T2 is a subtype of T1. This is based on the `Liskov Substitution Principle (LSP)`, which states that if an object of type T2 can replace an object of type T1 without altering any of the desirable properties of the program, then T2 is a subtype of T1. This is also known as behavioral subtyping


In this example, T2 is a subclass of T1, so you can use an instance of T2 wherever T1 is expected. However, the reverse is not true


In [ ]:
class T1: ...


class T2(T1): ...


def f1(p: T1) -> None: ...


o2 = T2()
f1(o2)  # OK


def f2(p: T2) -> None: ...


o1 = T1()
f2(o1)  # type error

Consistent-with:

1. Given T1 and a subtype T2, then T2 is consistent-with T1 (Liskov substitution).

2. Every type is consistent-with Any: you can pass objects of every type to an argument declared of type Any.

3. Any is consistent-with every type: you can always pass an object of type Any where an argument of another type is expected.


In [ ]:
from typing import Any


def f3(p: Any) -> None: ...


o0 = object()
o1 = T1()
o2 = T2()
f3(o0)  #
f3(o1)  # all OK: rule #2
f3(o2)  #


def f4():  # implicit type: `Any`
    ...


o4 = f4()  # inferred type: `Any`
f1(o4)  #
f2(o4)  # all OK: rule #3
f3(o4)  #

Inference: Modern type checkers in Python and other languages don’t require type annotations everywhere because they can infer the type of many expressions. For example, if you write x = len(s) \* 10, the type checker doesn’t need an explicit local declaration to know that x is an int, as long as it can find type hints for the len built-in.


## Simple Types and Classes


Simple types like int, float, str, and bytes can be used directly in type hints.

In the context of classes, the consistent-with relationship is defined similarly to the subtype-of relationship: a subclass is consistent-with all its superclasses.


There is no nominal subtype relationship between the built-in types int, float, and complex: they are direct subclasses of object. However, PEP 484 declares that int is consistent-with float, and float is consistent-with complex. This makes sense in practice: int implements all operations that float does, and int implements additional ones as well—bitwise operations like &, |, <<, etc. The end result is: int is consistent-with complex. For i = 3, i.real is 3, and i.imag is 0.


## Optional and Union types


The construct Optional[str] is actually a shortcut for Union[str, None], which
means the type of plural may be str or None.

In python 3.10, we can write `str | bytes` instead of `Union[str, bytes]` :

- Before - > `plural: Optional[str] = None `
- After - > `plural: str | None = None `


In [ ]:
from typing import Optional
from typing import Union


def show_count(count: int, singular: str, plural: Optional[str] = None) -> str: ...
def ord(c: Union[str, bytes]) -> int: ...
def parse_token(token: str) -> Union[str, float]:  # may return a str or a float:
    try:
        return float(token)
    except ValueError:
        return token


# Union[A, B, Union[C, D, E]] == Union[A, B, C, D, E]
# Not a good use case Union[int, float]  because int is consistent-with float.

## Generic Collections


In [ ]:
# Example 8-8. tokenize with type hints for Python ≥ 3.9
def tokenize(text: str) -> list[str]:  # returns a list where every item is of type str
    return text.upper().split()

The annotations `stuff: list` and `stuff: list[Any]` mean the same thing.

Collections from the standard library accepting generic type hints:\
`list, collections.deque, abc.Sequence, abc.MutableSequenceو set, abc.Container, abc.Set, abc.MutableSet, frozenset, abc.Collection`


$\color{red}{warning:}$
There is no good way to annotate array.array, taking into
account the typecode constructor argument, which determines whether integers or
floats are stored in the array. An even harder problem is how to type check integer
ranges to prevent OverflowError at runtime when adding elements to arrays.

$\color{green}{example:}$ if you try to add 256 to an array with typecode=‘B’, you will get an OverflowError. The type system cannot detect this error before running the code, because it does not know the value of 256, only that it is an integer.


## Tuple Types


- **Tuples as records**


In [ ]:
from geolib import geohash as gh  # type: ignore 'stops mypy from reporting that the geolib package doesn't have type hints'

PRECISION = 9


def geohash(lat_lon: tuple[float, float]) -> str:
    return gh.encode(*lat_lon, PRECISION)


shanghai = 31.2304, 121.4737
geohash(shanghai)

'wtw3sjq6q'

- **Tuples as records with named fields** <br>
  `typing.NamedTuple` is a factory for tuple subclasses, so Coordinate is _consistent-with_ <br> `tuple[float, float]` but **the reverse is not true** . <br>
  Also, Coordinate has extra methods added by NamedTuple, like `._asdict()`, and could also have user-defined methods.


In [ ]:
from typing import NamedTuple
from geolib import geohash as gh  # type: ignore

PRECISION = 9


class Coordinate(NamedTuple):
    lat: float
    lon: float


def geohash(lat_lon: Coordinate) -> str:
    return gh.encode(*lat_lon, PRECISION)

- **Tuples as immutable sequence**

  `tuple[int, ...]` is a tuple with int items. The ellipsis indicates that any number of elements >= 1 is acceptable. <br>
  There is no way to specify fields of different types for tuples of arbitrary length.


In [ ]:
# Example 8-13.
from collections.abc import Sequence


def columnize(sequence: Sequence[str], num_columns: int = 0) -> list[tuple[str, ...]]:
    if num_columns == 0:
        num_columns = round(len(sequence) ** 0.5)
    num_rows, reminder = divmod(len(sequence), num_columns)
    num_rows += bool(reminder)
    return [tuple(sequence[i::num_rows]) for i in range(num_rows)]


animals = "drake fawn heron ibex koala lynx tahr xerus yak zapus".split()
columnize(animals)

[('drake', 'koala', 'yak'),
 ('fawn', 'lynx', 'zapus'),
 ('heron', 'tahr'),
 ('ibex', 'xerus')]

## Generic Mappings


Generic mapping types are annotated as `MappingType[KeyType, ValueType]`


In [ ]:
# Example 8-14.
import sys
import re
import unicodedata
from collections.abc import Iterator

RE_WORD = re.compile(r"\w+")
STOP_CODE = sys.maxunicode + 1


def tokenize(text: str) -> Iterator[str]:
    """return iterable of uppercased words"""
    for match in RE_WORD.finditer(text):
        yield match.group().upper()


def name_index(start: int = 32, end: int = STOP_CODE) -> dict[str, set[str]]:
    index: dict[str, set[str]] = {}  # str as key set of str as value

    for char in (chr(i) for i in range(start, end)):
        if name := unicodedata.name(char, ""):
            for word in tokenize(name):
                index.setdefault(word, set()).add(char)
    return index

## Abstract Base class


Be conservative in what you send (do), be liberal in what you accept.

—Postel’s law, a.k.a. the Robustness Principle

using ABCs in type hints is better than using concrete types, because it gives more flexibility to the caller of the function

why ?

- Using Mapping[str, int] is better, because it allows the caller to provide any object that is a subclass of Mapping, such as dict, defaultdict, ChainMap, or a custom class that inherits from UserDict. This way, the function can accept different kinds of mapping objects, as long as they have the required methods and properties defined by the ABC

- Using dict[str, int], which is more restrictive, because it only accepts objects that are subclasses of dict, such as defaultdict or OrderedDict. This excludes some objects that are also valid mapping objects, such as a custom class that inherits from UserDict.

using ABCs in type hints is a way of being liberal in what you accept, because it allows more variations and diversity in the input types. However, using concrete types in type hints is a way of being conservative in what you send, because it ensures more consistency and precision in the output types.


In [ ]:
from collections.abc import Mapping


def name2hex(
    name: str, color_map: Mapping[str, int]
) -> str: ...  #   subtype-of Mapping.
def name2hex(
    name: str, color_map: dict[str, int]
) -> str: ...  # must be a dict or one of its subtypes,

The fall of the numeric tower

- Number
- Complex
- Real
- Rational
- Integral

In practice, if you want to annotate numeric arguments for static type checking, you
have a few options:

1. Use one of the concrete types int, float, or complex—as recommended by PEP
2. Declare a union type like Union[float, Decimal, Fraction].
3. If you want to avoid hardcoding concrete types, use numeric protocols like Supports Float, covered in “Runtime Checkable Static Protocols” on page 468.


## Iterable


- Iterable
  most useful ABCs for type hints: Iterable.


In [ ]:
from collections.abc import Iterable

FromTo = tuple[str, str]


def zip_replace(text: str, changes: Iterable[FromTo]) -> str:
    for from_, to in changes:
        text = text.replace(from_, to)
    return text


l33t = [("a", "4"), ("e", "3"), ("i", "1"), ("o", "0")]
text = "mad skilled noob powned leet"
zip_replace(text, l33t)

# from typing import TypeAlias
# FromTo: TypeAlias = tuple[str, str]

'm4d sk1ll3d n00b p0wn3d l33t'

**abc.Iterable vs abc.Sequence**

- `abc.Iterable` represents any object that can be iterated over, but it doesn't make any guarantees about the order of iteration or the ability to index elements.
- `abc.Sequence`, on the other hand represents a finite ordered set of elements. You can access elements by index, and it has a defined length.


## Parameterized Generics and TypeVar


A parameterized generic is a generic type, written as `list[T]`, where `T` is a type variable
that will be bound to a specific type with each usage.


In [ ]:
# Example 8-16. sample.py
from collections.abc import Sequence
from random import shuffle
from typing import TypeVar

T = TypeVar("T")


def sample(
    population: Sequence[T], size: int
) -> list[
    T
]:  # T could be any type—int, str, float, etc., In this case it returns the same type
    if size < 1:
        raise ValueError("size must be >= 1")
    result = list(population)
    shuffle(result)
    return result[:size]

In [1]:
# Example 8-17. mode_float.py: mode that operates on float and subtypes
from collections import Counter
from collections.abc import Iterable


def mode(data: Iterable[float]) -> float:
    pairs = Counter(data).most_common(1)
    if len(pairs) == 0:
        raise ValueError("no mode for empty data")
    return pairs[0][0]


mode([1, 1, 2, 3, 3, 3, 3, 4])

3

we see that the Counter class is used for
ranking. Counter is based on dict, therefore the element type of the data iterable
must be hashable. Now the problem is that the type of the returned item is Hashable: an ABC that
implements only the ** hash ** method. So the type checker will not let us do anything
with the return value except call hash() on it.

We need to restrict the
possible types assigned to T. Using **Restricted TypeVar** and **Bounded TypeVar**.


- **Restricted TypeVar**


A restricted type variable will be set to one of the types named in the TypeVar
declaration.


In [ ]:
# Restricted TypeVar
from collections.abc import Iterable
from decimal import Decimal
from fractions import Fraction
from typing import TypeVar

NumberT = TypeVar("NumberT", float, Decimal, Fraction)
NumberT = TypeVar(
    "NumberT", float, Decimal, Fraction, str
)  # this would make the name NumberT misleading

- **Bounded TypeVar**

  A bounded type variable will be set to the inferred type of the expression—as
  long as the inferred type is consistent-with the boundary declared in the bound=
  keyword argument of TypeVar.


`issue:` type checker will not let us do anything with the return value except call `hash()` on it.


In [ ]:
# Bounded TypeVar
# Example 8-18.
from collections import Counter
from collections.abc import Iterable, Hashable
from typing import TypeVar

HashableT = TypeVar("HashableT", bound=Hashable)  # hashable and any subtype of it


def mode(data: Iterable[HashableT]) -> HashableT:
    pairs = Counter(data).most_common(1)
    if len(pairs) == 0:
        raise ValueError("no mode for empty data")
    return pairs[0][0]

The typing module includes a predefined TypeVar named AnyStr that accept either bytes or str and return values of the given type.


In [ ]:
AnyStr = TypeVar("AnyStr", bytes, str)

# Static Protocols


- In Python, a protocol definition is written as a typing.Protocol subclass. However,
  classes that implement a protocol don’t need to inherit, register, or declare any relationship
  with the class that defines the protocol. It’s up to the type checker to find the
  available protocol types and enforce their usage.


In [ ]:
# Example 8-19 The problem is how to constrain T?
def top(series: Iterable[T], length: int) -> list[T]:
    ordered = sorted(series, reverse=True)
    return ordered[:length]


sorted([object() for _ in range(4)])

TypeError: '<' not supported between instances of 'object' and 'object'

In [ ]:
class Spam:  # we can use sorted() because of __lt__
    def __init__(self, n):
        self.n = n

    def __lt__(self, other):
        return self.n < other.n

    def __repr__(self):
        return f"Spam({self.n}) "


sorted([Spam(n) for n in range(5, 0, -1)])

[Spam(1) , Spam(2) , Spam(3) , Spam(4) , Spam(5) ]

A type T is consistent-with a protocol P if T implements all the methods defined in P,
with matching type signatures.


In [ ]:
from typing import Protocol, Any


class SupportsLessThan(Protocol):
    def __lt__(self, other: Any) -> bool: ...

In [ ]:
from collections.abc import Iterable
from typing import TypeVar

LT = TypeVar("LT", bound=SupportsLessThan)


def top(series: Iterable[LT], length: int) -> list[LT]:
    ordered = sorted(series, reverse=True)
    return ordered[:length]

The tests are designed to confirm that the top function works as expected with series of tuples, and raises a TypeError when the series contains plain object instances (which do not support the < operator).


In [ ]:
# Example 8-22.
from collections.abc import Iterator
from typing import TYPE_CHECKING
import pytest


def test_top_tuples() -> None:
    fruit = "mango pear apple kiwi banana".split()
    series: Iterator[tuple[int, str]] = ((len(s), s) for s in fruit)
    length = 3
    expected = [(6, "banana"), (5, "mango"), (5, "apple")]
    result = top(series, length)
    if TYPE_CHECKING:  # useful for code that only needs to be seen by the type checker and not executed at runtime
        reveal_type(series)
        reveal_type(expected)
        reveal_type(result)
    assert result == expected


# intentional type error
def test_top_objects_error() -> None:
    series = [object() for _ in range(4)]
    if TYPE_CHECKING:
        reveal_type(series)
    with pytest.raises(TypeError) as excinfo:
        top(series, 3)
    assert "'<' not supported" in str(excinfo.value)

The mypy output also shows a \* after any type that was inferred, indicating that the type was not explicitly annotated. The error flagged by mypy is intentional and confirms that the top function raises a TypeError when the series contains object instances.


Example 8-23. Output of mypy top_test.py (lines split for readability)

```
…/comparable/ $ mypy top_test.py
top_test.py:32: note:
Revealed type is "typing.Iterator[Tuple[builtins.int, builtins.str]]"

top_test.py:33: note:
Revealed type is "builtins.list[Tuple[builtins.int, builtins.str]]"

top_test.py:34: note:
Revealed type is "builtins.list[Tuple[builtins.int, builtins.str]]"

top_test.py:41: note:
Revealed type is "builtins.list[builtins.object*]"

top_test.py:43: error:
Value of type variable "LT" of "top" cannot be "object"

Found 1 error in 1 file (checked 1 source file)
```

Now we can make duck typing explicit for static type checkers. That’s why it makes
sense to say that typing.Protocol gives us static duck typing.


## Callable


A Callable type is parameterized likethis:
Callable[[ParamType1, ParamType2], ReturnType]


In [ ]:
from typing import Callable, Any


def repl(input_fn: Callable[[Any], str] = input) -> None: ...

- Covariance: If B is a subtype of A, then Container[B] can be treated as a subtype of Container[A]. This is safe for return types.
- Contravariance: If B is a subtype of A, then Container[A] can be treated as a subtype of Container[B]. This is safe for input types.

In the context of Callable, the return type is covariant, and the parameter types are contravariant. This means that if a function is expected to return a float (which is a supertype of int), it’s acceptable to provide a function that returns an int. However, if a function is expected to take a float as an argument, it’s not acceptable to provide a function that takes an int.

If a function is expected to return a float (which is a supertype of int), it’s acceptable to provide a function that returns an int. However, if a function is expected to take a float as an argument, it’s not acceptable to provide a function that takes an int
Formally, we say that Callable[[], int] is subtype-of Callable[[], float]

Most parameterized generic types are invariant, which means that they don’t allow covariance or contravariance. Formally, Callable[[int], None] is not a subtype-of Callable[[float], None].
Although int is subtype-of float, in the parameterized Callable type the relationship
is reversed: Callable[[float], None] is subtype-of Callable[[int], None].
Therefore we say that Callable is contravariant on the declared parameter types.


In [ ]:
# Example 8-24. Illustrating variance.
from collections.abc import Callable


def update(probe: Callable[[], float], display: Callable[[float], None]) -> None:
    temperature = probe()
    # imagine lots of control code here
    display(temperature)


def probe_ok() -> int:
    return 42


def display_wrong(temperature: int) -> None:
    print(hex(temperature))


update(probe_ok, display_wrong)  # type error


def display_ok(temperature: complex) -> None:
    print(temperature)
    update(probe_ok, display_ok)  # Ok

## NoReturn


A special type used only to annotate the return type of functions that never
return


In [ ]:
from typing import NoReturn


def exit(__status: object = ...) -> NoReturn: ...

## Annotating Positional Only and Variadic Parameters


In [ ]:
from typing import Optional


def tag(
    name: str,
    /,
    *content: str,
    class_: Optional[str] = None,
    **attrs: str,
) -> str:
    pass

\*content: str => all arguments must be of type str\
\*\*attrs: float => dict[str, float]


**Imperfect Typing and Strong Testing**

- Static Typing and Bug Detection: Static type checkers can help find and fix many bugs in large codebases more cheaply than if the bugs were discovered only after the code is running in production. However, automated testing was widely adopted long before static typing was introduced.

- Limitations of Static Typing: Static typing cannot be trusted as the ultimate arbiter of correctness. It can produce false positives (reporting type errors on correct code) and false negatives (not reporting type errors on incorrect code).

- Loss of Expressiveness: If we are forced to type check everything, we lose some of the expressive power of Python. Some handy features can’t be statically checked, like argument unpacking (config(\*\*settings)), and advanced features like properties, descriptors, metaclasses, and metaprogramming are poorly supported or beyond comprehension for type checkers.

- Lagging Behind Python Releases: Type checkers can lag behind Python releases, rejecting or even crashing while analyzing code with new language features.

- Inability to Express Common Data Constraints: Type hints are unable to ensure certain data constraints. For example, they can’t ensure “quantity must be an integer > 0” or “label must be a string with 6 to 12 ASCII letters.” In general, type hints are not helpful to catch errors in business logic.

- Type Hints and Software Quality: Given these caveats, type hints cannot be the mainstay of software quality, and making them mandatory without exception would amplify the downsides.

- Static Type Checker as a Tool in CI Pipeline: A static type checker should be considered as one of the tools in a modern Continuous Integration (CI) pipeline, along with test runners, linters, etc. The point of a CI pipeline is to reduce software failures, and automated tests catch many bugs that are beyond the reach of type hints.

- Strong Typing vs. Strong Testing: The title and conclusion of this section were inspired by Bruce Eckel’s article “Strong Typing vs. Strong Testing”. He was a static typing advocate until he learned Python and concluded that if a Python program has adequate unit tests, it can be as robust as a C++, Java, or C# program with adequate unit tests (although the tests in Python will be faster to write).


# Lecturers

1. Mahyar Jahaninasab [Github](https://github.com/mahyar-jahaninasab)
2. Ghazal Tajik [Linkedin](https://www.linkedin.com/in/ghazal-tajik-1736a126b)

present date :


# Reviewers

1. Fahimeh Asadi , review date: 2023-11-10, [LinkedIn](https://www.linkedin.com/in/fahimehasaadi/)
2. Mahya Asgarian , review date: 2023-11-10, [LinkedIn](https://www.linkedin.com/in/mahya-asgarian-9a7b13249/)
